In [26]:
import json, re
from collections import defaultdict

In [163]:
# 1000 normostran
HAHAKOEFICIENT = 1800 * 1000

In [164]:
def to_utc(date):
    def trnsl(month):
        months = ["ledna", "února", "března", "dubna", "května", "června", "července", "srpna", "září", "října", "listopadu", "prosince"]
        return months.index(month) + 1
    date_parts = date.split()
    return date_parts[2] + "-" \
        + str(trnsl(date_parts[1])).zfill(2) + "-" \
        + date_parts[0].strip(".").zfill(2)

In [165]:
with open("data.json", encoding="utf8") as file:
    smich_data = json.load(file)

In [166]:
for smich in smich_data:
    smich_count = len(re.findall("\(.*?[sS]mích.*?\)", smich["speech"]))
    smich["haha"] = smich_count
    smich["wc"] = len(smich["speech"])
    smich["date"] = to_utc(smich["date"])
    del smich["speaker"]
    del smich["link"]
    del smich["speech"]

In [167]:
# merge
data = {date: defaultdict(int) for date in set([el["date"] for el in smich_data])}
for smich in smich_data:
    for el in ["haha", "wc"]:
        data[smich["date"]][el] += smich[el]

In [168]:
# normalize
for date in data:
    data[date]["hahaIndex"] = round(data[date]["haha"] / data[date]["wc"] * HAHAKOEFICIENT)

In [169]:
with open("results_timeline.json","w",encoding="utf8") as file:
    json.dump(data, file, ensure_ascii=False)

In [170]:
with open("results_timeline_hahaindex.json","w",encoding="utf8") as file:
    json.dump(sorted([[date, data[date]["hahaIndex"]] for date in data]), file, ensure_ascii=False)